In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline   

In [2]:
data = pd.read_csv('./gifts-data.tsv', delimiter='\t')

#dropping invalid rows (every second BTW)
data = data.drop([i for i in range (1, len(data), 2)], axis=0)

#droppint working columns
to_drop = ['INPUT:theme', 'ASSIGNMENT:link', 'ASSIGNMENT:assignment_id', 'ASSIGNMENT:worker_id',
           'ASSIGNMENT:status', 'ASSIGNMENT:started', 'ACCEPT:verdict', 'ACCEPT:comment']
data = data.drop(to_drop, axis=1)

#dropping fully nan columns
for col in data.columns:
    nan_ = sum(data[col].isnull())
    if nan_ == len(data):
        data = data.drop([col], axis=1)

#checking out what's left
for key, value in data.iloc[0].iteritems():
    print (key, value)

OUTPUT:self_age 53.0
OUTPUT:self_time 2.0
OUTPUT:person_age 56.0
OUTPUT:person_tel 2.0
OUTPUT:person_auto 0.0
OUTPUT:person_hair 0.0
OUTPUT:person_time 7.0
OUTPUT:person_work 2.0
OUTPUT:self_gender 1.0
OUTPUT:person_hobby 0.0
OUTPUT:person_music 4.0
OUTPUT:person_sport 0.0
OUTPUT:person_gender 1.0
OUTPUT:person_height 3.0
OUTPUT:person_mental 0.0
OUTPUT:person_social 2.0
OUTPUT:self_variants 1.0
OUTPUT:person_animals 1.0
OUTPUT:self_gift_link nan
OUTPUT:self_gift_mark 0.0
OUTPUT:self_gift_name Аккустическая система с фм
OUTPUT:person_psycho_1 1.0
OUTPUT:person_psycho_2 0.0
OUTPUT:person_psycho_3 0.0
OUTPUT:person_psycho_4 0.0
OUTPUT:person_psycho_5 0.0
OUTPUT:person_psycho_6 0.0
OUTPUT:person_psycho_7 0.0
OUTPUT:person_psycho_8 1.0
OUTPUT:person_psycho_9 1.0
OUTPUT:self_gift_times 6.0
OUTPUT:self_gift_usage 0.0
OUTPUT:person_closeness 4.0
OUTPUT:person_education 1.0
OUTPUT:self_gift_desc_1 0.0
OUTPUT:self_gift_desc_2 0.0
OUTPUT:self_gift_desc_3 1.0
OUTPUT:self_gift_desc_4 1.0
OUTPUT:se

In [3]:
data.head(5)

,OUTPUT:self_age,OUTPUT:self_time,OUTPUT:person_age,OUTPUT:person_tel,OUTPUT:person_auto,OUTPUT:person_hair,OUTPUT:person_time,OUTPUT:person_work,OUTPUT:self_gender,OUTPUT:person_hobby,...,OUTPUT:self_gift_desc_13,OUTPUT:self_wish_to_give,OUTPUT:person_temperament,OUTPUT:person_animals_like,OUTPUT:self_gift_bad_usage,OUTPUT:person_character_word,OUTPUT:person_appearance_word,OUTPUT:person_description_text,OUTPUT:person_communication_word,OUTPUT:person_individuality_word
0,53.0,2.0,56.0,2.0,0.0,0.0,7.0,2.0,1.0,0.0,...,2.0,1.0,3.0,0.0,1.0,уравновешенный,приятная,"чистоплотный, организованный, ответственный, п...",приятное,порядочный
2,41.0,2.0,8.0,2.0,1.0,1.0,8.0,4.0,0.0,0.0,...,4.0,0.0,4.0,2.0,1.0,Весельчак.,Приятный.,"Неусидчивый, очень легко отвлекается и увлекае...",Ненавязчивое.,Наивный стяжатель.
4,30.0,1.0,30.0,0.0,1.0,2.0,7.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,реалист,миловидная,"Ум, скромность, честность, привлекательность, ...",приятное,ответственность
6,19.0,2.0,24.0,1.0,1.0,4.0,8.0,3.0,1.0,0.0,...,5.0,2.0,4.0,0.0,1.0,Веселая,Красивая,Немного наивная верит в любовь с первого взгля...,Легко,Уникальность
8,24.0,1.0,30.0,1.0,0.0,1.0,6.0,1.0,1.0,0.0,...,1.0,1.0,2.0,0.0,1.0,спокойный,красавчик,лучшая\tлегкая\tмилая\tмечтательная манящая ми...,любовное,характер


In [4]:
t = 340
data['OUTPUT:self_gift_name'].str.lower().str.replace('-', '').str.strip().unique()[t:t+20]

array(['корзинка для цветов', 'парфюм, косметика',
       'декоративная подушка с надписью.', 'свитер легкий',
       'блонот под заказ', 'косметика по уходу за лицом',
       'духи шанель номер 5', 'статуэтка', 'кухонные весы', 'зонтик',
       'альбом с фотографиями', 'игровые наушники',
       'принадлежности для ванны', 'зимняя шуба', 'собака',
       'лошадиный дилдо (нет серьезно, не шучу)', 'комнатные тапки',
       'книга "дачный дом..."', 'самодельное пирожное',
       'метакса 12 звезд'], dtype=object)

In [5]:
# drop 'джіпіес' 'эестетический' 'подарок' 'comfort' 'подарок от чистого сердца,сделанный своими руками'
mapping = {'смартфон': 'телефон',
           'колье':    'украшение',
           'часы casio': 'часы',
           'несколько шоколадок, его любимое печенье': 'шоколад, печенье',
           'францая духи названия не помню':'духи',
           'духи hugo boss':'духи',
           'духи hugo boss deep red':'духи',
            'часы наручные':'часы',
             'подарочная карта в магазин рив гош':'подарочная карта',
             'браслет':'украшение',
             'ручуа':'ручка',
             'картина пейзаж':'картина',
             'мобильный телефон bq stanford и парфюм': 'телефон',
             'это был планшет, сделанный на заказ.': 'планшет',
             'айфон': 'телефон',
              'парфюм ': 'духи',
              'ювелирное украшение': 'украшение',
              'парфюм': 'духи',
              'цепочка с кулоном': 'украшение',
              'сертификат в магазин косметики': 'подарочная карта',
              'это были чехол на паспорт, майка с рисунком волка, рюкзак': 'чехол на паспорт',
          'macbook pro': 'ноутбук',
           'маленькая мягкая игрушка': 'мягкая игрушка',
            'картина с его любимым автомобилем, по его фотографии, нарисованная мной': 'картина',
            'часы мужские дизель': 'часы',
            'sony xperia z': 'телефон',
            'оригинальная футболка fila': 'футболка',
            'смартчаси': 'часы',
            '2 теплых свитера ванс': 'свитер',
            'смарт часы': 'часы',
            'bluetooth наушник': 'наушники',
             'драгоценное украшение': 'украшение',
             'самсунг галакси': 'телефон',
             'игрушечная бетономешалка на пульте ду) . так как у моего парня бизнес с бетономешалками': 'машинка на пульте управления'
            }

In [7]:
def map_(x):
    if x in mapping.keys():
        return mapping[x]
    elif 'парфюм' in x or 'духи' in x or 'туалетная' in x:
        return 'духи'
    elif 'планшет' in x:
        return 'планшет'
    elif 'ноутбук' in x or 'нетбук' in x or 'тлефнр' in x:
        return 'ноутбук'
    elif 'украшение' in x or 'браслет' in x or 'кулон' in x or 'цепоч' in x \
        or 'подвеск' in x or 'золот' in x or 'серебр' in x or 'колечко' in x:
        return 'украшение'
    elif 'часы' in x or 'watch' in x:
        return 'часы'
    elif 'запонк' in x:
        return 'запонки'
    elif 'телефон' in x or 'смартфон' in x or 'phone' in x \
         or 'самсунг' in x or 'samsung' in x or 'xiaomi' in x:
        return 'телефон'
    elif 'мягк' in x or 'медв' in x:
        return 'мягкая игрушка'
    elif 'белье' in x:
        return 'белье'
    elif 'удочка' in x or 'спиннинг' in x or 'удилище' in x or 'спининг' in x:
        return 'удочка'
    elif 'игра' in x:
        return 'игра'
    elif 'наушник' in x:
        return 'наушники'
    elif 'крем' in x or 'космет' in x:
        return 'уходовая косметика'
    elif 'delongi' in x:
        return 'кофемашина'
    elif 'банк' in x:
        return 'павербанк'
    elif 'портрет' in x:
        return 'картина'
    elif 'колонк' in x:
        return 'колонка'
    elif 'котенок' in x or 'катенок' in x or 'кошка' in x or 'котёнок' in x:
        return 'кошка'
    else:
        return x
len(data['OUTPUT:self_gift_name'].str.lower().str.replace('-', '').str.strip().apply(map_).unique())

413

In [8]:
data['OUTPUT:self_gift_name'] = data['OUTPUT:self_gift_name'].str.lower().str.replace('-', '').str.strip().apply(map_)

In [9]:
data['OUTPUT:self_gift_name'].head(5)

0    аккустическая система с фм
2                  конструктор.
4                       телефон
6                     украшение
8                      наушники
Name: OUTPUT:self_gift_name, dtype: object

In [10]:
data.to_csv('gifts_cleared.tsv')

In [1]:
import pandas as pd

In [4]:
col = pd.read_csv('gifts_cleared_by_hands.tsv')['self_gift_name_cleared']

In [5]:
col.nunique()

344